In [1]:
import hyperspy.api as hs
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import os

In [2]:
##### SEM Apreo March 2023 #####
path = '../../Masteroppgave/2023-03-08_EDS-Apreo/exports/'

In [3]:
aztec_TEM_file = '../../Masteroppgave/2023-03-08_EDS-Apreo/AZtec quant as EDS_TEM.xlsx'
df = pd.read_excel(aztec_TEM_file, sheet_name=1)

In [4]:
def quant_as_TEM(filename, a_corr=True, noise_slice=0.65, print_results=False):
    # check if filename exists in path
    if filename + '.emsa' not in os.listdir(path):
        print(filename + '.emsa', 'not found in path')
        return
    
    df_s = df[df['File'] == filename]
    df_s = df_s.sort_values(by=['Element']) # alphabetically, as HS

    # make the spectrum and the model
    s = hs.load(path + filename + '.emsa', signal_type='EDS_TEM')
    elements = df_s['Element'].values
    s = s.isig[noise_slice:] # noise peak slice
    # if Vacc is lower than 20, slice at Vacc
    if s.metadata.Acquisition_instrument.TEM.beam_energy < 20:
        s = s.isig[:s.metadata.Acquisition_instrument.TEM.beam_energy]

    s.add_elements(elements)
    s.add_lines()
    m = s.create_model()
    m.fit()
    m.fit_background()

    lines = df_s['Line'].values
    kfactors = df_s['k-factor'].values
    intensities = s.get_lines_intensity()
    m_intensities = m.get_lines_intensity()

    # select only the relevant lines in m (relevant = we have a k-factor for it from AZtec)    
    lines_in_m_to_use = []
    for m_line in m_intensities:
        if m_line.metadata.Sample.xray_lines[0] in lines:
            lines_in_m_to_use.append(m_line)
    thickness_list = [1e3,1e4,1e5]
    if a_corr:
    # calculate %at for each line in m, if possible
        for l in lines:
            try:
                m[l]
                for thickness in thickness_list:
                    at_m = s.quantification(lines_in_m_to_use, factors=kfactors, method='CL', absorption_correction=True, thickness=thickness)
                    df_s[f'HS %at m A t={thickness:.0e}'] = [at_m[0][0].data.round(2)[0], at_m[0][1].data.round(2)[0]]
            except:
                print(l, 'not found in m, quantification with model not possible')
                df_s[f'HS %at m A t={thickness:.0e}'] = [np.nan, np.nan]

        # do the CL quantification with the intensities from the original spectrum
        at = s.quantification(intensities, factors=kfactors, method='CL', absorption_correction=True, thickness=thickness)
        df_s['HS %at s A'] = [at[0][0].data.round(2)[0], at[0][1].data.round(2)[0]]
    
    else: # no absorption correction
        # almost the same code, but slicing at and at_m with one less level
        for l in lines:
            try:
                m[l]
                at_m = s.quantification(lines_in_m_to_use, factors=kfactors, method='CL')
                df_s['HS %at m'] = [at_m[0].data.round(2)[0], at_m[1].data.round(2)[0]]
            except:
                print(l, 'not found in m, quantification with model not possible')
                df_s['HS %at m'] = [np.nan, np.nan]
        at = s.quantification(intensities, factors=kfactors, method='CL')
        df_s['HS %at s'] = [at[0].data.round(2)[0], at[1].data.round(2)[0]]
            

    if print_results:
        print(df_s)

    # put results back in df
    df[df['File'] == filename] = df_s

In [5]:
df.head(1)

,File,Element,Line series,k-factor,Line,AZ %at,HS %at s,HS %at s A,HS %at m,HS %at m A t=1e+03,HS %at m A t=1e+04,HS %at m A t=1e+05
0,GaAs_05kV_25pA,As,L,1.21,As_La,45.4,0,0,0,0.0,0.0,0


In [6]:
quant_as_TEM('GaAs_30kV_25pA',print_results=True)

Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 104.46 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 107.39 ms


Absorption correction calculation: 0it [00:00, ?it/s]

...

[########################################] | 100% Completed | 121.32 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 121.84 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 105.82 ms
             File Element Line series  k-factor   Line  AZ %at  HS %at s  \
6  GaAs_30kV_25pA      As           K     4.191  As_Ka   42.03         0   
7  GaAs_30kV_25pA      Ga           K     3.268  Ga_Ka   57.97         0   

   HS %at s A  HS %at m  HS %at m A t=1e+03  HS %at m A t=1e+04  \
6       62.03         0               42.63               49.93   
7       37.97         0               57.37               50.07   

   HS %at m A t=1e+05  
6               61.36  
7               38.64  


In [7]:
quant_as_TEM('GaAs_30kV_25pA',print_results=True, a_corr=False)


[########################################] | 100% Completed | 119.19 ms
[########################################] | 100% Completed | 111.45 ms
[########################################] | 100% Completed | 115.87 ms
             File Element Line series  k-factor   Line  AZ %at  HS %at s  \
6  GaAs_30kV_25pA      As           K     4.191  As_Ka   42.03     42.54   
7  GaAs_30kV_25pA      Ga           K     3.268  Ga_Ka   57.97     57.46   

   HS %at s A  HS %at m  HS %at m A t=1e+03  HS %at m A t=1e+04  \
6       62.03     41.52               42.63               49.93   
7       37.97     58.48               57.37               50.07   

   HS %at m A t=1e+05  
6               61.36  
7               38.64  


In [8]:
quant_as_TEM('GaAs_30kV_25pA',print_results=True)
quant_as_TEM('GaAs_30kV_25pA',print_results=True, a_corr=False)



Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 107.20 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 116.63 ms


Absorption correction calculation: 0it [00:00, ?it/s]

...

[########################################] | 100% Completed | 120.89 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 115.86 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 118.20 ms
             File Element Line series  k-factor   Line  AZ %at  HS %at s  \
6  GaAs_30kV_25pA      As           K     4.191  As_Ka   42.03     42.54   
7  GaAs_30kV_25pA      Ga           K     3.268  Ga_Ka   57.97     57.46   

   HS %at s A  HS %at m  HS %at m A t=1e+03  HS %at m A t=1e+04  \
6       62.03     41.52               42.63               49.93   
7       37.97     58.48               57.37               50.07   

   HS %at m A t=1e+05  
6               61.36  
7               38.64  
[########################################] | 100% Completed | 106.13 ms
[########################################] | 100% Completed | 112.74 ms
[########################################] | 100% Completed | 105.15 ms
             File Element Line series  k-factor   Line  AZ %at  HS %at s  \
6  GaAs_30kV_25pA      As           K     4.191  As_Ka   42.03     42.54   
7  GaAs_30kV_25pA      Ga           K     3.268  Ga_Ka   

In [9]:
# took 3-4 min
for f in df['File'].unique():
    print(f)
    quant_as_TEM(f)

GaAs_05kV_25pA


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 113.28 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 115.17 ms


...

[########################################] | 100% Completed | 121.87 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 108.70 ms


Absorption correction calculation: 0it [00:00, ?it/s]

[########################################] | 100% Completed | 117.54 ms
Map1
Map1.emsa not found in path
Map2
Map2.emsa not found in path


In [10]:
# took 30 sec
for f in df['File'].unique():
    print(f)
    quant_as_TEM(f, a_corr=False)

GaAs_05kV_25pA
[########################################] | 100% Completed | 115.32 ms
[########################################] | 100% Completed | 128.82 ms
[########################################] | 100% Completed | 120.77 ms
GaAs_10kV_25pA
[########################################] | 100% Completed | 114.57 ms
[########################################] | 100% Completed | 128.48 ms
[########################################] | 100% Completed | 132.24 ms
GaAs_15kV_25pA
[########################################] | 100% Completed | 108.74 ms
[########################################] | 100% Completed | 119.68 ms
[########################################] | 100% Completed | 120.16 ms
GaAs_30kV_25pA
[########################################] | 100% Completed | 111.70 ms
[########################################] | 100% Completed | 116.57 ms
[########################################] | 100% Completed | 109.97 ms
GaAs_30kV_50pA
[########################################] | 100% Completed |

In [11]:
df.head(3)

,File,Element,Line series,k-factor,Line,AZ %at,HS %at s,HS %at s A,HS %at m,HS %at m A t=1e+03,HS %at m A t=1e+04,HS %at m A t=1e+05
0,GaAs_05kV_25pA,As,L,1.210,As_La,45.40,45.00,62.93,45.41,59.51,63.19,63.19
1,GaAs_05kV_25pA,Ga,L,1.086,Ga_La,54.60,55.00,37.07,54.59,40.49,36.81,36.81
2,GaAs_10kV_25pA,As,L,1.310,As_La,42.09,41.43,60.58,41.62,56.79,60.71,60.71


In [12]:
y_keys = ['AZ %at', 'HS %at s', 'HS %at s A', 'HS %at m', 'HS %at m A t=1e+03', 'HS %at m A t=1e+04', 'HS %at m A t=1e+05']
x_key = ['File']



In [13]:
# save the df to excel
# df.to_excel('results/quant_as_TEM.xlsx')

In [14]:
fig = go.Figure()
lines_list = ['Sb_La', 'As_Ka', 'Ga_Ka', 'Ga_La', 'As_La']
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52'] # plotly colors

for l in lines_list:
    for y in y_keys:
        i = y_keys.index(y)
        fig.add_scatter(x=df[df['Line'] == l]['File'], y = df[df['Line'] == l][y], mode='markers', name=y, marker=dict(size=8, color=colors[i]), legendgroup=y, showlegend=True if l == lines_list[0] else False)
fig.update_layout(title='Data treated as TEM, quantified with HS using different quantification parameters<br><sub>s=original spectrum, m=model, A=absorption correction, t=thickness')
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.update_yaxes(range=[0, 100])
fig

In [15]:
fig.write_html('results/quant_as_TEM.html')